#Retrieval-Augmented Generation (RAG) QA Bot
This notebook demonstrates the implementation of a Retrieval-Augmented Generation (RAG) model for a Question-Answering (QA) bot using OpenAI API and Pinecone. The bot retrieves relevant information from a vector database and generates natural language answers.

## Key Features:
- Stores business-related documents in a Pinecone vector database.
- Retrieves the most relevant documents based on user queries.
- Generates human-like answers using OpenAI's GPT model.


In [15]:
!pip install pinecone-client
!pip install openai==0.27.0

## Setup and Dependencies
Before running this notebook, ensure you have:
- A valid Pinecone API key and environment.
- An OpenAI API key.
- Installed the required libraries (`pinecone`, `openai`).

This section installs and initializes the required libraries.


In [25]:
import openai
from pinecone import Pinecone, ServerlessSpec

In [40]:
PINECONE_API_KEY = "your_pinecone_API_key"
PINECONE_ENVIRONMENT = "pinecone_environment"
OPENAI_API_KEY = "your_OpenAI_API_key"

openai.api_key = OPENAI_API_KEY

## Pinecone Setup
Pinecone is a vector database used to store embeddings of documents. We initialize Pinecone with our API key and create an index to manage the data.

Key steps:
- Connect to Pinecone using the API key.
- Create an index for storing document embeddings.
- Ensure the index supports the required dimensionality and metric type.


In [27]:
# Initialize Pinecone instance
pc = Pinecone(api_key=PINECONE_API_KEY)

In [28]:
INDEX_NAME = "qa-bot-index"
DIMENSION = 1536


# Connect to the index
index = pc.Index(name=INDEX_NAME)


In [29]:
# Function to create text embeddings
def embed_text(text):
    response = openai.Embedding.create(input=text, model="text-embedding-ada-002")
    return response['data'][0]['embedding']


## Document Preparation
This section demonstrates how to add documents to the Pinecone index. Each document contains an `id` and `content`. The content is embedded using OpenAI's text-embedding model and stored in the vector database.

Steps:
- Create a list of documents relevant to the business domain.
- Generate embeddings using OpenAI API.
- Upsert (insert or update) the embeddings into the Pinecone index.


In [30]:
# Example documents to upsert
documents = [
    {"id": "doc1", "content": "We specialize in AI solutions."},
    {"id": "doc2", "content": "Our office is in Mumbai."},
    {"id": "doc3", "content": "We offer cloud computing services."},
    {"id": "doc4", "content": "Customer support is available 24/7."},
    {"id": "doc5", "content": "Our products include chatbots, recommendation engines, and sentiment analysis tools."},
    {"id": "doc6", "content": "We have been recognized as a leader in AI innovation."},
    {"id": "doc7", "content": "Our mission is to make AI accessible to small businesses."},
    {"id": "doc8", "content": "We support integrations with leading cloud platforms like AWS, Azure, and Google Cloud."},
    {"id": "doc9", "content": "Our CEO, Manish Kumar, has 15 years of experience in AI and machine learning."},
]

# Upsert documents to Pinecone index
for doc in documents:
    embedding = embed_text(doc["content"])
    index.upsert([{
        "id": doc["id"],
        "values": embedding,
        "metadata": {"content": doc["content"]}
    }])


## Answer Generation
OpenAI's GPT model is used to generate answers by combining the retrieved document content and the user query. This enhances the bot's ability to provide contextually relevant and accurate answers.

Key points:
- Combine retrieved document data with user queries.
- Generate responses using OpenAI GPT models.


In [31]:
# Function to retrieve top matching documents
def retrieve_documents(query, top_k=3):
    query_embedding = embed_text(query)
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )
    return [result['metadata']['content'] for result in results['matches']]

In [42]:
# Function to generate a meaningful response using GPT
def generate_response(query, documents):

    # Combine the document content into a single string
    context = " ".join(documents)

    # Creating the prompt for GPT
    prompt = f"""
    You are a helpful assistant for a business QA bot. Use the context provided below to answer the user's question.

    Context: {context}

    User Query: {query}

    Answer:
    """

    # Generating the response using GPT
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=150
    )

    # Returning the generated answer
    return response['choices'][0]['message']['content'].strip()


## Testing the QA Bot
Use example queries to test the bot's performance. Ensure the retrieved documents and generated answers align with the business context.

Example query:
"What renewable energy solutions do you provide?"


In [43]:
# Example query
query = "what kind of products do you offer? And what about the customer service?"

# Retrieve top-matching documents
matches = retrieve_documents(query)
print("Top Matches:", matches)

# Generate a meaningful response
response = generate_response(query, matches)
print("Generated Response:", response)


Top Matches: ['Customer support is available 24/7.', 'Our products include chatbots, recommendation engines, and sentiment analysis tools.', 'We offer cloud computing services.']
Generated Response: We offer a variety of products including chatbots, recommendation engines, and sentiment analysis tools. Additionally, we provide cloud computing services. In relation to customer service, it is available 24/7 to assist with any needs or inquiries you may have.


## Conclusion
This notebook demonstrates the integration of Pinecone and OpenAI to build a RAG-based QA bot. By combining vector search with language generation, the bot provides precise and human-like answers to user queries.


